In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import DoubleType, LongType,IntegerType, DateType



In [2]:
spark = SparkSession.builder.appName("ArticleTransaction").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/06/10 00:29:07 WARN Utils: Your hostname, Abhijit resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/10 00:29:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 00:29:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [82]:
# Read CSV files with inferred schema and header
articles_df = spark.read.csv("../article.csv", header=True, inferSchema=False)
transactions_df = spark.read.csv("../transactions.csv", header=True, inferSchema=False)

## Article DF inital exploration

In [ ]:
print("Articles DataFrame:")
articles_df.show(5)
print("Articles DataFrame Schema:")
articles_df.printSchema()
print("Count of Articles DataFrame:", articles_df.count())
print("Unique Article IDs in Articles DataFrame:", articles_df.select("ARTICLE_ID0MATERIAL").distinct().count())
# 4.1 Check for duplicate keys in ARTICLE
dup_articles = articles_df.groupBy("ARTICLE_ID0MATERIAL").count().filter(col("count") > 1)
print("Duplicate ARTICLE_ID0MATERIAL rows:", dup_articles.count())

print("Is Article ID having null values?", articles_df.filter(col("ARTICLE_ID0MATERIAL").isNull()).count() > 0)
print("Is Article ID having empty values?", articles_df.filter(col("ARTICLE_ID0MATERIAL") == "").count() > 0)
print("Is Article ID having empty null strings?", articles_df.filter(col("ARTICLE_ID0MATERIAL") == "null").count() > 0)
# print("Article df statistics:")
# articles_df.describe().show()

Articles DataFrame:
+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
|ARTICLE_ID0MATERIAL|ARTICLE_COLOR_ID0RT_COLOR|ARTICLE_GROUP_ID0RT_CONFMAT|EAN0EANUPC|   DESCRIPTION0TXTMD|BRAND_NAME/SOL/MDPROD1|PICTURE_PATH0EXT_URL|INITIAL_SEASON_NAME/SOL/FMSSEASO|CURRENT_SEASON_NAME|MATL_TYPE0MATL_TYPE|
+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
| 000000000001282247|                     null|                       null|      null|d3dd68a11f543d039...|                    75|                null|                          202008|             202008|               ZMO3|
| 000000000001282474|                     null|                       null|     

### I can observe ARTICLE_ID0MATERIAL is long. Lot of column values are having "null" string which can be converted to actual null values. There are no duplicate rows in article dataframe. Lot of column can make use of datatype change.

In [109]:
# Replace "null" strings with actual nulls
cleaned_articles_df = articles_df.replace("null", None)

#Type conversion
cleaned_articles_df = (
    cleaned_articles_df
      # numeric identifiers → LongType
      .withColumn("ARTICLE_ID0MATERIAL",   col("ARTICLE_ID0MATERIAL").cast(LongType()))
      .withColumn("ARTICLE_GROUP_ID0RT_CONFMAT", col("ARTICLE_GROUP_ID0RT_CONFMAT").cast(LongType()))
      .withColumn("EAN0EANUPC", col("EAN0EANUPC").cast(LongType()))
      .withColumn("BRAND_NAME/SOL/MDPROD1",     col("BRAND_NAME/SOL/MDPROD1").cast(IntegerType()))
      .withColumn("INITIAL_SEASON_NAME/SOL/FMSSEASO",     col("INITIAL_SEASON_NAME/SOL/FMSSEASO").cast(IntegerType()))
      .withColumn("CURRENT_SEASON_NAME",     col("CURRENT_SEASON_NAME").cast(IntegerType()))

)

cleaned_articles_df.show(5)

+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
|ARTICLE_ID0MATERIAL|ARTICLE_COLOR_ID0RT_COLOR|ARTICLE_GROUP_ID0RT_CONFMAT|EAN0EANUPC|   DESCRIPTION0TXTMD|BRAND_NAME/SOL/MDPROD1|PICTURE_PATH0EXT_URL|INITIAL_SEASON_NAME/SOL/FMSSEASO|CURRENT_SEASON_NAME|MATL_TYPE0MATL_TYPE|
+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
|            1282247|                     NULL|                       NULL|      NULL|d3dd68a11f543d039...|                    75|                NULL|                          202008|             202008|               ZMO3|
|            1282474|                     NULL|                       NULL|      NULL|d6045a9b489ed9

## Transaction DF initial exploration

In [108]:
print("Transaction DataFrame:")
transactions_df.show(5,truncate=False)
# print("Transaction DataFrame Schema:")
# transactions_df.printSchema()
print("Count of Transaction DataFrame:", transactions_df.count())
print("Unique TRANSACTION_ID in Transaction DataFrame:", transactions_df.select("TRANSACTION_ID/SOL/BONKEY").distinct().count())
# 4.1 Check for duplicate keys in ARTICLE
dup_transaction = transactions_df.groupBy("TRANSACTION_ID/SOL/BONKEY").count().filter(col("count") > 1)
print("Duplicate TRANSACTION_ID/SOL/BONKEY rows:", dup_transaction.count())
dup_transaction = transactions_df.groupBy(["TRANSACTION_ID/SOL/BONKEY","ARTICLE_ID0MATERIAL"]).count().filter(col("count") > 1)
print("Duplicate TRANSACTION_ID/SOL/BONKEY, ARTICLE_ID0MATERIAL rows:", dup_transaction.count())
print("Unique TRANSACTION_ID, Article ID in Transaction DataFrame:", transactions_df.select(["TRANSACTION_ID/SOL/BONKEY", "ARTICLE_ID0MATERIAL"]).distinct().count())

print("Is Article ID having null values?", transactions_df.filter(col("ARTICLE_ID0MATERIAL").isNull()).count() > 0)
print("Is Article ID having empty values?", transactions_df.filter(col("ARTICLE_ID0MATERIAL") == "").count() > 0)
print("Is Article ID having empty null strings?", transactions_df.filter(col("ARTICLE_ID0MATERIAL") == "null").count() > 0)

print("Is Transaction ID having null values?", transactions_df.filter(col("TRANSACTION_ID/SOL/BONKEY").isNull()).count() > 0)
print("Is Transaction ID having empty values?", transactions_df.filter(col("TRANSACTION_ID/SOL/BONKEY") == "").count() > 0)
print("Is Transaction ID having empty null strings?", transactions_df.filter(col("TRANSACTION_ID/SOL/BONKEY") == "null").count() > 0)

Transaction DataFrame:
+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|TRANSACTION_ID/SOL/BONKEY|TRANSACTION_TIME0RPA_ETS2|SALESORG0SALESORG|TRANSACTION_DATE0CALDAY|DISTR_CHAN0DISTR_CHAN|ARTICLE_ID0MATERIAL|LOCATION_ID0PLANT|TRANSACTION_TYPE0RPA_TTC|ARTICLE_COUNT0RPA_RLQ|SALES_PRICE_AT_CASH_DESK0RPA_SAT|SALES_PRICE_PLANNED/SOL/LOC0086C|VAT0RPA_TAM|
+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|0002021040620240000000061|121315                   |1099             |20210406               |13                   |00000000206204202

### After analysis I found that the data does not have any duplicate rows. It has a composite key (TRANSACTION_ID/SOL/BONKEY and ARTICLE_ID0MATERIAL). The data type of the coulmns needs to be defined explicitly.

In [ ]:
# Replace "null" strings with actual nulls
cleaned_transactions_df = transactions_df.replace("null", None)

#Type conversion
cleaned_transactions_df = (
    cleaned_transactions_df
      # numeric identifiers → LongType
      .withColumn("ARTICLE_ID0MATERIAL",   col("ARTICLE_ID0MATERIAL").cast(LongType()))
      .withColumn("ARTICLE_GROUP_ID0RT_CONFMAT", col("ARTICLE_GROUP_ID0RT_CONFMAT").cast(LongType()))
      .withColumn("EAN0EANUPC", col("EAN0EANUPC").cast(LongType()))
      .withColumn("BRAND_NAME/SOL/MDPROD1",     col("BRAND_NAME/SOL/MDPROD1").cast(IntegerType()))
      .withColumn("INITIAL_SEASON_NAME/SOL/FMSSEASO",     col("INITIAL_SEASON_NAME/SOL/FMSSEASO").cast(IntegerType()))
      .withColumn("CURRENT_SEASON_NAME",     col("CURRENT_SEASON_NAME").cast(IntegerType()))

)

cleaned_transactions_df.show(5)

+-------------------------+-----+
|TRANSACTION_ID/SOL/BONKEY|count|
+-------------------------+-----+
|0002021040620990000000016|2    |
|0002021040861250000000007|2    |
|0002021040620240000000139|2    |
|0002021040844050000000076|3    |
|0002021040720390000000157|3    |
|0002021040722300000000010|3    |
|0002021040820610000000003|3    |
|0002021040760970000000074|2    |
|0002021040820810000000009|2    |
|0002021040641280000000002|2    |
|0002021040878180000000052|2    |
|0002021040720240000000117|13   |
|0002021040840210000000033|3    |
|0002021040622300000000053|2    |
|0002021040777870000000036|5    |
|0002021040663390000000016|3    |
|0002021040673370000000104|2    |
|0002021040720390000000009|4    |
|0002021040721110000000020|5    |
|0002021040622310000000047|3    |
+-------------------------+-----+
only showing top 20 rows

root
 |-- TRANSACTION_ID/SOL/BONKEY: string (nullable = true)
 |-- count: long (nullable = false)



In [80]:
dup_transaction_1 = transactions_df.filter(transactions_df['TRANSACTION_ID/SOL/BONKEY'] == '0002021040720240000000117' )
dup_transaction_1.show( truncate=False)

+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|TRANSACTION_ID/SOL/BONKEY|TRANSACTION_TIME0RPA_ETS2|SALESORG0SALESORG|TRANSACTION_DATE0CALDAY|DISTR_CHAN0DISTR_CHAN|ARTICLE_ID0MATERIAL|LOCATION_ID0PLANT|TRANSACTION_TYPE0RPA_TTC|ARTICLE_COUNT0RPA_RLQ|SALES_PRICE_AT_CASH_DESK0RPA_SAT|SALES_PRICE_PLANNED/SOL/LOC0086C|VAT0RPA_TAM|
+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|0002021040720240000000117|155608                   |1099             |20210407               |13                   |000000002060679001 |2024             |10

In [22]:
spark.stop()